# Tabu Search

We will be implementing our second meta-heuristic algorithm for the Travelling Salesman Problem (TSP) called Tabu Search. This algorithm is memory-based and is an iterative approach to the problem which works by removing components and marking them as Tabu as it iterates over the solution thus creating a neighbourhood which is restricted to only use non-tabu components.



Pseudocode: (Tabu Search)

Algorithm - Tabu Search
Input: TabuList Size
Output: Sbest (Shortest cycle path)


1.  TabuList = ∅
2.  while (stopCondition())
3.    candidateList ← ∅
4.    for (Scandidate ∈ Sbestneighbourhood)
5.     if (containsAnyFeatures(Scandidate, TabuList))
6.       candidateList ← Scandidate
7.     end if
8.  end while
9.  Scandidate ← LocateBestCandidate(candidateList)
10. if(Cost(Scandidate)≤ Cost(Sbest))
11.  Sbest ← Scandidate
12.  TabuList ← FeatureDifferences(Scandidate, Sbest)
13.  while(TabuList > TabuListsize)
14.    DeleteFeature(TabuList)
15.  end while
16. end if
17. Return (Sbest)

(Brownlee, 2015)


References:

http://www.cleveralgorithms.com/nature-inspired/stochastic/tabu_search.html (Accessed 26th March 2020)


Big O-Notation:






In [20]:
from random import randint, choice
from pprint import pprint
from itertools import permutations
from math import inf as oo
from math import sqrt
from time import time
import matplotlib.pyplot as plt
import copy

MAX_DISTANCE = 100

def random_symmetric_graph(n):
    ''' Symmetric adjacency matrix of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            v = randint(1,MAX_DISTANCE)
            dist_matrix[i][j] = v
            dist_matrix[j][i] = v
    return dist_matrix

def random_euclidean_graph(n):
    ''' Symmetric adjacency matrix of a Euclidean graph of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    points = []
    for p in range(n):
        x,y = randint(0,MAX_DISTANCE), randint(0,MAX_DISTANCE)
        points.append((x,y))
    for i in range(n):
        p1 = points[i]
        for j in range(i+1,n):
            p2 = points[j]
            distance = sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
            dist_matrix[i][j] = distance
            dist_matrix[j][i] = distance
    return dist_matrix

def show(G):
    ''' Show adjacency matrix. Useful for debugging. '''
    n = len(G)
    r = "     "
    for i in range(n):
        r += f'{i:4}'
    r += '\n    -'+'-'*(4*n)+'\n'
    for i in range(n):
        r += f'{i:2} | '
        for j in range(n):
            r += f'{G[i][j]:4}'
        r += '\n'
    r = r.replace('inf', '  ∞')
    print(r)

def cost(G, cycle):
    ''' Calculate the cost of the given cycle '''
    c = 0
    n = len(G)
    for i in range(n):
        a = cycle[i]
        b = cycle[(i+1)%n]
        c += G[a][b]
    return c

In [28]:
G = random_symmetric_graph(12)

In [30]:
show(G)

        0   1   2   3   4   5   6   7   8   9  10  11
    -------------------------------------------------
 0 |    ∞  77   9  69  27  94  94  87   5  24  77  95
 1 |   77   ∞  64  32  70  48  17  80   8   9  16  25
 2 |    9  64   ∞   4  52  26  75  96  80  77  36  30
 3 |   69  32   4   ∞   2  73  48  54  33  28  96  65
 4 |   27  70  52   2   ∞  19  54  63  42  32  36  28
 5 |   94  48  26  73  19   ∞  53  88 100  89  95  10
 6 |   94  17  75  48  54  53   ∞  38  13  47  76  58
 7 |   87  80  96  54  63  88  38   ∞  29  91   7  16
 8 |    5   8  80  33  42 100  13  29   ∞  49   7  28
 9 |   24   9  77  28  32  89  47  91  49   ∞  50  72
10 |   77  16  36  96  36  95  76   7   7  50   ∞  33
11 |   95  25  30  65  28  10  58  16  28  72  33   ∞



# Tabu Search

In [27]:
import networkx as graphs
import random
from random import shuffle
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class WeightedGraph:
    n = 0
    p = 0
    low_weight = 0
    up_weight = 0
    distmatrix = {}
    w_edges = []
    def __init__(self,n,p,low_weight,up_weight):
        """
        Variable n: number of nodes
        Variable p: The probability of two nodes becoming connected
        low/up weight: Possible weight values
        """
        self.n = n
        self.p = p
        self.low_weight= low_weight
        self.up_weight = up_weight
        
    def RandomWGraph(self):
        g = graphs.gnp_random_graph(self.n,self.p)
        m = g.number_of_edges()
        weights = [random.randint(self.low_weight, self.up_weight) for r in range(m)]
        #unweighted connections
        uw_edges = g.edges()
        # Create weighted graph edge list
        i=0
        w_edges = []
        ret_graph = graphs.Graph()
        for edge in uw_edges:
        #w_edges = [uw_edges[i][0], uw_edges[i][1], weights[i]]
        #w_edges+={(edge[0],edge[1]):weights[i]}
            ret_graph.add_edge(edge[0],edge[1],weight=weights[i])
            i =i +1
        #print(w_edges)
        #return graphs.Graph(w_edges, weighted = True,s=weights)
        return ret_graph

In [28]:
def graphdata(nodes, probability, min_weight, max_weight):
    linkset = []
    links = {}

    if min_weight>max_weight:
        print('Lower weight cannot be greater then upper weight for the weight range. ')
        sys.exit()
    if probability<0 or probability>1:
        print('Probability incorrect. Must be between 0 and 1. ')
        sys.exit()
    generated_data = WeightedGraph(nodes, probability, min_weight, max_weight)
    generated = generated_data.RandomWGraph()
    node_list=list(generated.nodes())
    weight_of_all_nodes=0
    for a in node_list:
        for b in node_list:
            if a==b:
                continue
            link = []
            link.append(a)
            link.append(b)
            weight_of_edge=generated.get_edge_data(a,b)['weight']
            link.append(weight_of_edge)
            linkset.append(link)
            print('%d %d %d' % (a,b,weight_of_edge))
            if weight_of_edge>weight_of_all_nodes:
                weight_of_all_nodes=weight_of_edge

    for link in linkset:
        try:
            linklist = links[str(link[0])]
            linklist.append(link[1:])
            links[str(link[0])] = linklist
        except:
            links[str(link[0])] = [link[1:]]

    return links, weight_of_all_nodes

In [29]:
def tabu_search(nodes, probability, min_weight, max_weight):
    global max_fitness, start_node
    graph, max_weight = graphdata(nodes, probability, min_weight, max_weight)

    ## Below, get the keys (node names) and shuffle them, and make start_node as start
    s0 = list(graph.keys())
    shuffle(s0)

    if int(s0[0]) != start_node:
        for i in range(len(s0)):
            if  int(s0[i]) == start_node:
                swap = s0[0]
                s0[0] = s0[i]
                s0[i] = swap
                break;

    # max_fitness will act like infinite fitness
    max_fitness = ((max_weight) * (len(s0)))+1
    sBest = s0
    vBest = (s0, graph)
    bestCandidate = s0
    tabuList = []
    tabuList.append(s0)
    stop = False
    best_keep_turn = 0

    start_time = time.time()
    while not stop :
        sNeighborhood = (bestCandidate)
        bestCandidate = sNeighborhood[0]
        for sCandidate in sNeighborhood:
            if (sCandidate not in tabuList) and (((sCandidate, graph) < (bestCandidate, graph))):
                bestCandidate = sCandidate


        tabuList.append(bestCandidate)
        if (len(tabuList) > maxTabuSize):
            tabuList.pop(0)

        if best_keep_turn == stoppingTurn:
            stop = True

        best_keep_turn += 1

    exec_time =  time.time() - start_time
    return sBest, vBest, exec_time



## Tabu Search Takes edge-list in a given format:
#nodefrom nodeto weight
#0 1 5
#3 2 4
#1 0 3
#Undirectional edges should be written 2 times for both nodes.
maxTabuSize = 10000
neighborhood_size = 500
stoppingTurn = 500
max_fitness = 0
start_node = 0
solution, value, exec_time = tabu_search(nodes=5, probability=1, min_weight=10, max_weight=20)

print(solution)
print('----> '.join(a for a in solution))
print('Shortest Distance Below:')
print(value)
print('Solved in %s seconds'%exec_time)

0 1 18
0 2 14
0 3 12
0 4 10
1 0 18
1 2 14
1 3 18
1 4 20
2 0 14
2 1 14
2 3 10
2 4 14
3 0 12
3 1 18
3 2 10
3 4 17
4 0 10
4 1 20
4 2 14
4 3 17
['0', '2', '1', '4', '3']
0----> 2----> 1----> 4----> 3
Shortest Distance Below:
(['0', '2', '1', '4', '3'], {'0': [[1, 18], [2, 14], [3, 12], [4, 10]], '1': [[0, 18], [2, 14], [3, 18], [4, 20]], '2': [[0, 14], [1, 14], [3, 10], [4, 14]], '3': [[0, 12], [1, 18], [2, 10], [4, 17]], '4': [[0, 10], [1, 20], [2, 14], [3, 17]]})
Solved in 0.00024509429931640625 seconds


References:

Graph Generation
https://gist.github.com/RobertTalbert/9f0879e5ed4b4297fc5f

Tabu Search Algorithm/GraphData
https://github.com/polatbilek/Tabu-search-on-Travelling-Salesman-Problem/blob/master/tabu_search.py 
